## Notebook 6: Ocean Color and Sea Surface Temperature

**The ocean isn't just blue - from space we can see where life thrives.**

When you look at the ocean from a plane, you might notice patches of different colors - turquoise near the coast, deep blue offshore, and sometimes greenish bands. These aren't random variations. They reveal the invisible world of **phytoplankton** - microscopic plants that form the base of the entire ocean food chain.

Satellites measure **ocean color** to estimate chlorophyll concentration, which tells us where phytoplankton are blooming. They also measure **sea surface temperature (SST)**, which affects marine life distribution and ocean circulation.

In this notebook, we'll:

- Visualize chlorophyll-a concentration in the Gulf of Mexico
- Map sea surface temperature patterns
- Compare summer vs. winter conditions
- See how the Mississippi River creates a productivity hotspot

This is how scientists monitor ocean health, track harmful algal blooms, and study the impacts of climate change on marine ecosystems.

## Setup

Same initialization as previous notebooks.

In [1]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
from google.cloud import storage

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Ready!


## What is ocean color?

**Ocean color** refers to the subtle variations in how the ocean reflects sunlight. Pure open-ocean water appears deep blue because water absorbs red light and scatters blue. But when phytoplankton are present, they add green to the mix.

**Chlorophyll-a** is the key measurement:
- It's the primary photosynthetic pigment in phytoplankton
- Higher chlorophyll = more phytoplankton = greener water
- Measured in **mg/m³** (milligrams per cubic meter)

**Why does this matter?**
- Phytoplankton produce ~50% of Earth's oxygen through photosynthesis
- They're the base of the marine food chain (phytoplankton → zooplankton → fish → whales)
- High productivity areas = good fishing grounds
- Too much = harmful algal blooms (red tides)

**What drives phytoplankton blooms?**
- **Nutrients**: Nitrogen, phosphorus from rivers, upwelling, or mixing
- **Light**: Sunlight for photosynthesis
- **Temperature**: Affects growth rates and stratification

The Mississippi River is a major nutrient source, making the northern Gulf of Mexico one of the most productive (and problematic) coastal areas in the US.

## Define area of interest: Gulf of Mexico

The Gulf of Mexico is a semi-enclosed sea bordered by the US, Mexico, and Cuba. It receives freshwater and nutrients from major rivers including the Mississippi, the largest river system in North America.

In [3]:
# Gulf of Mexico bounding box
gulf_of_mexico = ee.Geometry.Rectangle([-98, 18, -80, 31])

# Key locations for reference
landmarks = {
    "Mississippi Delta": [-89.5, 29.0],
    "New Orleans": [-90.07, 29.95],
    "Tampa Bay": [-82.5, 27.8],
    "Corpus Christi": [-97.4, 27.8],
    "Cancun": [-86.85, 21.16],
}

# Center point for maps
center_lat = 25.5
center_lon = -89

print("Area of interest: Gulf of Mexico")
print("Key feature: Mississippi River plume brings nutrients into the Gulf")

Area of interest: Gulf of Mexico
Key feature: Mississippi River plume brings nutrients into the Gulf


## Load MODIS Ocean Color data

NASA's MODIS-Aqua satellite has been measuring ocean color since 2002. The **L3SMI** (Level 3 Standard Mapped Image) product provides global daily composites of chlorophyll-a concentration.

We'll compare two seasons:
- **Winter (Jan-Feb)**: Strong mixing brings nutrients up from deep water
- **Summer (Jul-Aug)**: Warmer, more stratified - Mississippi plume effects more visible

In [4]:
# MODIS Ocean Color - Chlorophyll-a concentration
ocean_color = ee.ImageCollection("NASA/OCEANDATA/MODIS-Aqua/L3SMI")

# Use 2023 for recent data
year = 2023

# Winter composite (January-February)
winter_chl = (
    ocean_color
    .filterDate(f"{year}-01-01", f"{year}-02-28")
    .select("chlor_a")  # Chlorophyll-a concentration in mg/m³
    .median()
    .clip(gulf_of_mexico)
)

# Summer composite (July-August)
summer_chl = (
    ocean_color
    .filterDate(f"{year}-07-01", f"{year}-08-31")
    .select("chlor_a")
    .median()
    .clip(gulf_of_mexico)
)

print(f"Loaded chlorophyll-a data for winter and summer {year}")

Loaded chlorophyll-a data for winter and summer 2023


## Visualize chlorophyll concentration

Chlorophyll-a values vary over orders of magnitude:
- **< 0.1 mg/m³**: Ultra-oligotrophic (nutrient-poor) open ocean - deep blue
- **0.1 - 1 mg/m³**: Typical open ocean
- **1 - 10 mg/m³**: Productive coastal and upwelling areas
- **> 10 mg/m³**: Very high productivity, potential algal blooms

We use a logarithmic color scale because of this wide range.

In [5]:
# Chlorophyll palette: purple (low) → blue → green → yellow (high)
chl_palette = [
    '#2c0a4a',  # Dark purple - very low
    '#4a1486',  # Purple
    '#0d47a1',  # Dark blue
    '#1976d2',  # Blue
    '#00acc1',  # Cyan
    '#00897b',  # Teal
    '#43a047',  # Green
    '#7cb342',  # Light green
    '#c0ca33',  # Yellow-green
    '#fdd835',  # Yellow
    '#ff8f00',  # Orange - high
]

# Use log scale for chlorophyll (0.01 to 30 mg/m³)
# Taking log10: log10(0.01) = -2, log10(30) ≈ 1.5
chl_log = summer_chl.log10()

chl_vis = {
    'min': -2,  # 0.01 mg/m³
    'max': 1.5, # ~30 mg/m³
    'palette': chl_palette
}

# Create map
m = geemap.Map(center=[center_lat, center_lon], zoom=5)

# Add chlorophyll layer
m.addLayer(chl_log, chl_vis, f'Chlorophyll-a Summer {year}')

# Add landmarks
for name, coords in landmarks.items():
    point = ee.Geometry.Point(coords)
    m.addLayer(point, {'color': 'white'}, name)

# Add region boundary
m.addLayer(gulf_of_mexico, {'color': 'white'}, 'Gulf of Mexico', False)

m.add_colorbar(
    chl_vis, 
    label='Chlorophyll-a (log10 mg/m³)', 
    layer_name=f'Chlorophyll-a Summer {year}'
)

m

## Load Sea Surface Temperature

Sea surface temperature (SST) is measured by both MODIS thermal bands and specialized ocean sensors. We'll use MODIS SST from the same collection for consistency.

SST affects:
- **Marine life distribution**: Many species have temperature preferences
- **Ocean stratification**: Warm surface water doesn't mix with cold deep water
- **Hurricane intensity**: Warm water (>26°C) fuels tropical storms
- **Coral health**: Prolonged high temps cause coral bleaching

In [6]:
# Load SST from MODIS Ocean Color product
# sst band is in degrees Celsius

# Winter SST
winter_sst = (
    ocean_color
    .filterDate(f"{year}-01-01", f"{year}-02-28")
    .select("sst")  # Sea surface temperature in °C
    .median()
    .clip(gulf_of_mexico)
)

# Summer SST
summer_sst = (
    ocean_color
    .filterDate(f"{year}-07-01", f"{year}-08-31")
    .select("sst")
    .median()
    .clip(gulf_of_mexico)
)

print(f"Loaded SST data for winter and summer {year}")

Loaded SST data for winter and summer 2023


In [7]:
# Thermal palette for SST: blue (cold) to red (hot)
sst_palette = [
    '#313695',  # Dark blue - cold
    '#4575b4',  
    '#74add1',  
    '#abd9e9',  
    '#e0f3f8',  # Light blue
    '#fee090',  # Yellow
    '#fdae61',  # Orange
    '#f46d43',  
    '#d73027',  # Red
    '#a50026',  # Dark red - hot
]

sst_vis = {
    'min': 18,   # 18°C
    'max': 32,   # 32°C
    'palette': sst_palette
}

# Create SST map
m2 = geemap.Map(center=[center_lat, center_lon], zoom=5)

m2.addLayer(summer_sst, sst_vis, f'Sea Surface Temperature Summer {year}')

# Add landmarks
for name, coords in landmarks.items():
    point = ee.Geometry.Point(coords)
    m2.addLayer(point, {'color': 'black'}, name)

m2.add_colorbar(
    sst_vis, 
    label='SST (°C)', 
    layer_name=f'Sea Surface Temperature Summer {year}'
)

m2

## Compare seasonal patterns

Let's compare winter vs. summer chlorophyll to see how productivity changes through the year.

**What to look for:**
- **Winter**: More mixing → nutrients reach surface → higher offshore productivity
- **Summer**: Warmer, stratified → Mississippi plume more concentrated along coast

In [8]:
# Create log-scale chlorophyll for both seasons
winter_chl_log = winter_chl.log10()
summer_chl_log = summer_chl.log10()

# Split map: Winter on left, Summer on right
left_layer = geemap.ee_tile_layer(winter_chl_log, chl_vis, f'Winter {year}')
right_layer = geemap.ee_tile_layer(summer_chl_log, chl_vis, f'Summer {year}')

m3 = geemap.Map(center=[center_lat, center_lon], zoom=5)
m3.split_map(left_layer, right_layer)
m3

## The Mississippi River plume

The Mississippi River drains 40% of the continental United States. It carries:
- **Sediment**: Makes water turbid (brown)
- **Nutrients**: Nitrogen and phosphorus from agricultural runoff
- **Freshwater**: Less dense than saltwater, so it spreads on top

This nutrient-rich freshwater creates a productivity hotspot - but also a problem. Excess nutrients cause massive algal blooms. When the algae die, bacteria decompose them and consume all the oxygen, creating a **"dead zone"** where fish and other marine life cannot survive.

The Gulf of Mexico dead zone is the largest in the US, covering 6,000-8,000 square miles in a typical summer.

Let's zoom in on the Mississippi Delta to see this effect.

In [9]:
# Zoom into Mississippi Delta region
delta_region = ee.Geometry.Rectangle([-92, 27, -87, 30.5])

# Create focused map on the delta
m4 = geemap.Map(center=[28.8, -89.5], zoom=7)

# Clip chlorophyll to delta region
delta_chl_log = summer_chl.clip(delta_region).log10()

m4.addLayer(delta_chl_log, chl_vis, f'Chlorophyll-a Summer {year}')

# Add delta landmark
delta_point = ee.Geometry.Point([-89.5, 29.0])
m4.addLayer(delta_point, {'color': 'red'}, 'Mississippi Delta')

m4.add_colorbar(
    chl_vis, 
    label='Chlorophyll-a (log10 mg/m³)', 
    layer_name=f'Chlorophyll-a Summer {year}'
)

print("Notice the high chlorophyll (yellow/green) spreading from the delta!")
print("This nutrient-rich plume can extend hundreds of kilometers.")
m4

Notice the high chlorophyll (yellow/green) spreading from the delta!
This nutrient-rich plume can extend hundreds of kilometers.


## Chlorophyll and SST: Side-by-Side

Is there a relationship between temperature and productivity?

- Cooler waters (from upwelling or river input) often have **higher** nutrients
- The Loop Current brings warm Caribbean water into the Gulf - typically **lower** productivity
- Coastal areas with river input show both cooler temps and higher chlorophyll

In [10]:
# Side-by-side: Chlorophyll (left) vs SST (right)
left_layer = geemap.ee_tile_layer(summer_chl_log, chl_vis, f'Chlorophyll-a')
right_layer = geemap.ee_tile_layer(summer_sst, sst_vis, f'SST')

m5 = geemap.Map(center=[center_lat, center_lon], zoom=5)
m5.split_map(left_layer, right_layer)

print("Drag the slider to compare chlorophyll (left) with temperature (right)")
print("Look for: cooler coastal water with high chlorophyll near the Mississippi")
m5

Drag the slider to compare chlorophyll (left) with temperature (right)
Look for: cooler coastal water with high chlorophyll near the Mississippi


## What patterns do you notice?

Looking at the maps:

**Chlorophyll patterns:**
- **Highest productivity** near the Mississippi Delta - the river plume is clearly visible
- **Coastal enhancement** along Louisiana, Texas, and Florida coasts
- **Low productivity** in the deep central Gulf (oligotrophic blue water)
- **Moderate levels** along the Yucatan coast (upwelling zones)

**Temperature patterns:**
- **Warmest water** (>30°C) in the central Gulf - the Loop Current brings Caribbean water
- **Cooler water** near the coast where river water mixes in
- **Seasonal variation** is dramatic - winter temps 10-15°C cooler than summer

**The connection:**
- River plumes are often cooler AND more productive
- The warm Loop Current is a productivity desert (nutrients are depleted)
- Eddies spinning off the Loop Current can entrain coastal water and nutrients

## Try it yourself

Ideas to explore:

1. **Harmful algal blooms (red tides)**: Florida's west coast experiences red tides from *Karenia brevis*. Look at chlorophyll during October-November 2017 or 2018 near Tampa Bay.

2. **El Niño effects**: Compare chlorophyll in a strong El Niño year (2015-2016) vs. a La Niña year (2020-2021). El Niño affects circulation patterns globally.

3. **Hurricane impacts**: Look at before/after a major hurricane. Storms mix the water column and can trigger plankton blooms. Try Hurricane Katrina (Aug 2005) or Hurricane Ida (Aug 2021).

4. **Other ocean basins**:
   - California Current: `ee.Geometry.Rectangle([-130, 30, -115, 45])` - major upwelling zone
   - Arabian Sea: `ee.Geometry.Rectangle([55, 10, 75, 25])` - monsoon-driven productivity
   - Southern Ocean: `ee.Geometry.Rectangle([-180, -70, 180, -50])` - phytoplankton blooms visible from space

5. **Time series**: Create monthly chlorophyll composites for a year to see the seasonal cycle of productivity.

6. **Coral bleaching**: Look at SST during the 2005 or 2023 bleaching events in the Caribbean. Temps above 29°C for extended periods cause coral stress.